<a href="https://colab.research.google.com/github/SoraHoang/Classification-of-Echocardiographic-Images/blob/main/ClassificationOfEchocardiographicImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thêm các thư viện**

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

# **Sử dụng data đã được lưu trên google drive**

In [ ]:
train_path = './drive/MyDrive/Data/DATA_CHAMBER_2021/train'
test_path = './drive/MyDrive/Data/DATA_CHAMBER_2021/test'

# **Chuẩn bị dữ liệu**

* Tạo namedtuple TrainTest cho tiện thao tác

In [ ]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

* 3 classes: {2C, 3C, 4C}

In [ ]:
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

Chuẩn bị dữ liệu

* Đọc dữ liệu từ đường dẫn train_path và test_path
* Kích thước không đồng bộ => ảnh raw cần rescale 224x224x3 (các ảnh scale sẽ có kích thước nhỏ hơn)
* Đưa dữ liệu ảnh về dạng tensor => 3ximage_resizeximage_resize
* Tùy theo các xử lý dữ liệu mà sẽ cho preprocess hoặc augmentation

In [ ]:
def choose_transform(image_resize=224, input_type='normal'):
  # size = {224, 64, 32}
  # type = {'normal', 'preprocess', 'augmentation'}
  if input_type == 'normal':
    transform_train = transforms.Compose([
      transforms.Resize((image_resize, image_resize)),
      transforms.ToTensor()
    ])
    transform_test = transforms.Compose([
      transforms.Resize((image_resize, image_resize)),
      transforms.ToTensor()
    ])
  elif input_type == 'preprocess':
    transform_train = transforms.Compose([
      transforms.Resize((image_resize, image_resize)),
      transforms.RandomEqualize(p=1),
      transforms.GaussianBlur(kernel_size=3),
      transforms.ToTensor()
    ])
    transform_test = transforms.Compose([
      transforms.Resize((image_resize, image_resize)),
      transforms.RandomEqualize(p=1),
      transforms.GaussianBlur(kernel_size=3),
      transforms.ToTensor()
    ])
  elif input_type == 'augmentation':
    transform_train = transforms.Compose([
      transforms.Resize((image_resize, image_resize)),
      transforms.RandomHorizontalFlip(p=0.3),
      transforms.RandomVerticalFlip(p=0.3),
      transforms.RandomRotation(degrees=10),
      transforms.ToTensor()
    ])
    transform_test = transforms.Compose([
      transforms.Resize((image_resize, image_resize)),
      transforms.ToTensor()
    ])
  return transform_train, transform_test

def prepare_data(transform_train, transform_test):
  trainset = torchvision.datasets.ImageFolder(root=train_path, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=test_path, transform=transform_test)

  return TrainTest(train=trainset, test=testset)

Chuẩn bị loader để chia batch dữ liệu

*   batch_size = 32
*   num_workers = 4



In [ ]:
def prepare_loader(datasets):
  batch_size = 32
  num_workers = 4

  trainloader = DataLoader(dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  return TrainTest(train=trainloader, test = testloader)

# **Xử lý các epoch**

Train trong mỗi epoch

In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  final_loss = 0.0
  reporting_steps = 60
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    final_loss = loss.item()
    running_loss += final_loss
    if i % reporting_steps == reporting_steps - 1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return final_loss

Test trong mỗi epoch

In [ ]:
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

# **Xây dựng và thực nghiệm mô hình**

In [ ]:
def main(MODEL, image_resize, input_type):
  PATH = './' + MODEL + '_' + str(image_resize) + '_' + input_type + '.pth'
  classes = get_classes()
  num_classes = len(classes)

  transform_train, transform_test = choose_transform(image_resize, input_type)
  datasets = prepare_data(transform_train, transform_test)

  loaders = prepare_loader(datasets)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  if MODEL == 'vgg16':
    model = torchvision.models.vgg16()

    num_features = model.classifier[6].in_features
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=num_features, out_features=num_classes)
  elif MODEL == 'resnet50':
    model = torchvision.models.resnet50()

    num_features = model.fc.in_features
    model.fc = torch.nn.modules.linear.Linear(in_features=num_features, out_features=num_classes)
  elif MODEL == 'densenet121':
    model = torchvision.models.densenet121()

    num_features = model.classifier.in_features
    model.classifier = torch.nn.modules.linear.Linear(in_features=num_features, out_features=num_classes)
  else:
    pass

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  accuracies = []
  losses = []
  for epoch in range(10):
    loss = train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    losses.append(round(loss, 4))

    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

    ypred = np.array(ypred)
    ytrue = np.array(ytrue)

    accuracy = int((ytrue == ypred).sum() / len(ytrue) * 100)
    accuracies.append(accuracy)

  print("model:", MODEL, ", size:", image_resize, ", type:", input_type)
  print("accuracy: ", accuracies)
  print("loss: ", losses)
  return model

# Thử mô hình

Test mô hình đã huấn luyện trên video

In [ ]:
def video_report(model, testsets):
  video = namedtuple('video', ['id', "label_true", 'label_pred'])
  path = []

  for index, image in enumerate(testsets.imgs):
    path.append(image[0].split("/")[-1].split("_")[0])
  
  vid_list = []
  for frame in path:
    if (frame in vid_list) == False:
      vid_list.append(frame)
    else:
      pass

  ytrue = []
  ypred = []
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  with torch.no_grad():
    model.eval()
    for image, label in testsets:
      image = image.unsqueeze(0).to(device)
      output = model(image)
      _, predicted = torch.max(output, dim=1)
      ytrue.append(label)
      ypred += list(predicted.cpu().numpy())

  outputs_vid = []
  ytrue_video = []
  ypred_video = []
  for vid_id in vid_list:
    vid_true = []
    vid_pred = []
    for index, img in enumerate(path):
      if img == vid_id:
        vid_true.append(ytrue[index])
        vid_pred.append(ypred[index])

    rate_video_true = [0, 0, 0] # vector count for voting
    rate_video_pred = [0, 0, 0]
    for label in range(3):
      rate_video_true[label] = list(vid_true).count(label)
      rate_video_pred[label] = list(vid_pred).count(label)
    
    label_video_true = np.argmax(rate_video_true)
    label_video_pred = np.argmax(rate_video_pred)

    print("id:", vid_id, ", true:", label_video_true, ", pred:",label_video_pred) # label pred
    ytrue_video.append(label_video_true)
    ypred_video.append(label_video_pred)
    outputs_vid.append(video(id=vid_id, label_true=label_video_true, label_pred=label_video_pred))
  print(classification_report(ytrue_video, ypred_video, target_names=get_classes()))
  return outputs_vid

In [ ]:
models = ['vgg16', 'resnet50', 'densenet121']
sizes = [224, 64, 32]
input_types = ['normal', 'preprocess', 'augmentation']

In [ ]:
model = main('densenet121', 224, 'augmentation')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 step 59 ave_loss 1.0597
Epoch 0 step 119 ave_loss 0.7584
Epoch 0 step 179 ave_loss 0.6047
              precision    recall  f1-score   support

          2C       0.68      0.81      0.74       409
          3C       0.48      0.77      0.59       367
          4C       1.00      0.63      0.77       831

    accuracy                           0.71      1607
   macro avg       0.72      0.74      0.70      1607
weighted avg       0.80      0.71      0.72      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 step 59 ave_loss 0.3428
Epoch 1 step 119 ave_loss 0.2713
Epoch 1 step 179 ave_loss 0.2698
              precision    recall  f1-score   support

          2C       0.65      0.71      0.68       409
          3C       0.40      0.87      0.55       367
          4C       0.91      0.39      0.55       831

    accuracy                           0.58      1607
   macro avg       0.65      0.66      0.59      1607
weighted avg       0.73      0.58      0.58      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Epoch 7 step 59 ave_loss 0.0322
Epoch 7 step 119 ave_loss 0.0399
Epoch 7 step 179 ave_loss 0.0162
              precision    recall  f1-score   support

          2C       0.66      0.81      0.73       409
          3C       0.68      0.88      0.76       367
          4C       1.00      0.76      0.86       831

    accuracy                           0.80      1607
   macro avg       0.78      0.82      0.79      1607
weighted avg       0.84      0.80      0.81      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 8 step 59 ave_loss 0.0186
Epoch 8 step 119 ave_loss 0.0197
Epoch 8 step 179 ave_loss 0.0216
              precision    recall  f1-score   support

          2C       0.63      0.82      0.71       409
          3C       0.63      0.87      0.73       367
          4C       1.00      0.69      0.81       831

    accuracy                           0.76      1607
   macro avg       0.75      0.79      0.75      1607
weighted avg       0.82      0.76      0.77      1607



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 step 59 ave_loss 0.0201
Epoch 9 step 119 ave_loss 0.0177
Epoch 9 step 179 ave_loss 0.0096
              precision    recall  f1-score   support

          2C       0.58      0.80      0.67       409
          3C       0.63      0.81      0.71       367
          4C       0.96      0.65      0.77       831

    accuracy                           0.72      1607
   macro avg       0.72      0.75      0.72      1607
weighted avg       0.79      0.72      0.73      1607

model: densenet121 , size: 224 , type: augmentation
accuracy:  [71, 58, 71, 48, 65, 79, 79, 79, 76, 72]
loss:  [0.3119, 0.2021, 0.0685, 0.0082, 0.1915, 0.0088, 0.0469, 0.1097, 0.0021, 0.0011]


In [ ]:
test_transform = transforms.Compose([
  transforms.Resize((224, 224)),
  # transforms.RandomEqualize(p=1),
  # transforms.GaussianBlur(kernel_size=3),
  transforms.ToTensor()
])

testsets = torchvision.datasets.ImageFolder(root=test_path, transform=test_transform)
video_report(model=model, testsets=testsets)

id: 158 , true: 0 , pred: 1
id: 165 , true: 0 , pred: 0
id: 168 , true: 0 , pred: 1
id: 169 , true: 0 , pred: 1
id: 171 , true: 0 , pred: 2
id: 176 , true: 0 , pred: 0
id: 177 , true: 0 , pred: 1
id: 178 , true: 0 , pred: 0
id: 181 , true: 0 , pred: 0
id: 183 , true: 0 , pred: 1
id: 191 , true: 0 , pred: 0
id: 192 , true: 0 , pred: 0
id: 157 , true: 1 , pred: 1
id: 159 , true: 1 , pred: 1
id: 161 , true: 1 , pred: 1
id: 162 , true: 1 , pred: 1
id: 166 , true: 1 , pred: 1
id: 174 , true: 1 , pred: 1
id: 175 , true: 1 , pred: 1
id: 179 , true: 1 , pred: 0
id: 185 , true: 1 , pred: 1
id: 186 , true: 1 , pred: 1
id: 189 , true: 1 , pred: 1
id: 190 , true: 1 , pred: 1
id: 194 , true: 1 , pred: 1
id: 160 , true: 2 , pred: 2
id: 163 , true: 2 , pred: 2
id: 164 , true: 2 , pred: 2
id: 167 , true: 2 , pred: 2
id: 170 , true: 2 , pred: 2
id: 172 , true: 2 , pred: 2
id: 173 , true: 2 , pred: 2
id: 180 , true: 2 , pred: 2
id: 182 , true: 2 , pred: 2
id: 184 , true: 2 , pred: 2
id: 187 , true: 2 , 

[video(id='158', label_true=0, label_pred=1),
 video(id='165', label_true=0, label_pred=0),
 video(id='168', label_true=0, label_pred=1),
 video(id='169', label_true=0, label_pred=1),
 video(id='171', label_true=0, label_pred=2),
 video(id='176', label_true=0, label_pred=0),
 video(id='177', label_true=0, label_pred=1),
 video(id='178', label_true=0, label_pred=0),
 video(id='181', label_true=0, label_pred=0),
 video(id='183', label_true=0, label_pred=1),
 video(id='191', label_true=0, label_pred=0),
 video(id='192', label_true=0, label_pred=0),
 video(id='157', label_true=1, label_pred=1),
 video(id='159', label_true=1, label_pred=1),
 video(id='161', label_true=1, label_pred=1),
 video(id='162', label_true=1, label_pred=1),
 video(id='166', label_true=1, label_pred=1),
 video(id='174', label_true=1, label_pred=1),
 video(id='175', label_true=1, label_pred=1),
 video(id='179', label_true=1, label_pred=0),
 video(id='185', label_true=1, label_pred=1),
 video(id='186', label_true=1, lab